This notebook implements a computation used in the proof of Theorem 6.4 of "Geometric decomposition of abelian varieties of order 1".

In [9]:
import itertools

This subroutine computes the mod 2 reduction of an element of a cyclotomic number field by reducing the coefficients in the power basis expansion. The field $K$ only needs to be specified if it cannot be read off from the parent of the element.

In [10]:
def mod2_cyc(t, K=None): 
    if K is None: 
        K = t.parent() 
    return K([i%2 for i in K(t).list()]) 

This subroutine tests whether or not a list of roots of unity contains a proper nonempty subsequence which is a mod 2 cyclotomic relation.

In [11]:
def contains_relation(l, K=None): 
    for i in range(1, len(l)-1): 
        for s in itertools.combinations(l, i): 
            if mod2_cyc(sum(s), K=K) == 0: 
                return True 
    return False

Compute the dictionary `d` defined as follows: for $t$ a cyclotomic integer and $i=2,\dots,6$, `d[t][i]` is a list of all sets of $i$ powers of $\zeta_{15}$ with sum congruent to $t$ mod 2 which do not contain a mod 2 cyclotomic relation.

In [12]:
d = {}
K.<z> = CyclotomicField(15)
for i in range(2, 7):
    for s in itertools.combinations(range(15), i):
        s1 = [z^j for j in s]
        if contains_relation(s1, K):
            continue
        t = mod2_cyc(sum(s1))
        if t not in d:
            d[t] = {}
            for j in range(2, 7):
                d[t][j] = []
        d[t][i].append(s)

Compute the dictionary `d2` defined as follows: for $t = 1 + \zeta_N$ with $N \in \{3,5,15\}$, for $t_2 \in \{1 \pm \zeta_N\}$, `d2[t][t2]` is a list of all sets of $i$ powers of $\zeta_{15}$ which can be lifted (by multiplying each term by a sign) to a list with sum $t_2$. In the process, we confirm that every entry of $d$ lifts to east one of the two options for $t_2$.

In [13]:
d2 = {}
for i0 in [1, 3, 5]:
    t = 1 + z^i0
    t1 = [1 + z^i0, 1-z^i0]
    d2[t] = {i: [] for i in t1}
    for i in range(2, 7):
        for s in d[t][i]:
            s1 = [z^j for j in s]
            abort = True
            for s2 in itertools.product([-1,1], repeat=i):
                t2 = sum(s1[j]*s2[j] for j in range(i))
                if t2 in t1:
                    d2[t][t2].append(s)
                    abort = False
            if abort:
                raise RuntimeError(t, s)

For each choice of $t$ and, find the size of the smallest sets of $i$ powers of $\zeta_{15}$ with sum congruent to $t$ mod 2 which do not lift to sets with sums $1+\zeta_N$ and $1-\zeta_N$.

In [15]:
for i0 in [1,3,5]:
    t = 1 + z^i0
    t1 = list(d2[t].keys())
    diff1 = [s for s in d2[t][t1[0]] if s not in d2[t][t1[1]]]
    diff2 = [s for s in d2[t][t1[1]] if s not in d2[t][t1[0]]]
    if not diff1 or not diff2:
        print("{}: No counterexample possible: {} {}".format(t, len(diff1), len(diff2)))
    else:
        print("{}: {}, {}".format(t, min(len(s) for s in diff1), min(len(s) for s in diff2)))

z + 1: 5, 4
z^3 + 1: No counterexample possible: 16 0
z^5 + 1: No counterexample possible: 19 0
